In [22]:
import sqlite3
import pandas as pd
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


df_games = pd.read_csv("originales/games.csv")
df_teams = pd.read_csv("originales/teams.csv")
df_teamstats = pd.read_csv("originales/teamstats.csv")
df_leagues = pd.read_csv("originales/leagues.csv")
df_appearances = pd.read_csv("originales/appearances.csv")
df_players = pd.read_csv("originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("originales/shots.csv")



""" df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")
df_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")
df_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")
df_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")
df_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")
df_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")
df_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") """

' df_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/games.csv")\ndf_teams = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teams.csv")\ndf_teamstats = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/teamstats.csv")\ndf_leagues = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/leagues.csv")\ndf_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/appearances.csv")\ndf_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/players.csv", encoding="ISO-8859-1")\ndf_shots = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/originales/shots.csv") '

In [23]:
df_teamstats_away = df_teamstats.loc[df_teamstats["location"] == "a"]
df_teamstats_home = df_teamstats.loc[df_teamstats["location"] == "h"]
df_teamstats_merged = pd.merge(
    df_teamstats_home, df_teamstats_away, on="gameID", suffixes=["_home", "_away"]
)

df_games_merged_with_teamstats = pd.merge(
    df_games, df_teamstats_merged, on="gameID", suffixes=("", "_remove")
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    df_games_merged_with_teamstats.iloc[:, 13:34], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["homeProbability", "drawProbability", "awayProbability"], axis=1
)
df_games_merged_with_teamstats = df_games_merged_with_teamstats.drop(
    ["teamID_home", "teamID_away"], axis=1
)

df_games_merged_with_leagues = pd.merge(
    df_games_merged_with_teamstats, df_leagues, on="leagueID"
)

df_games_merged_with_team = pd.merge(
    df_games_merged_with_leagues,
    df_teams,
    left_on=["homeTeamID"],
    right_on=["teamID"],
    how="left",
)
df_games_merged_with_team = pd.merge(
    df_games_merged_with_team,
    df_teams,
    left_on=["awayTeamID"],
    right_on=["teamID"],
    how="left",
)

df_f_game = df_games_merged_with_team.rename(
    columns={"name_x": "liga", "name_y": "home_name", "name": "away_name"}
)
df_f_game = df_f_game.drop(
    [
        "understatNotation",
        "season_home",
        "date_home",
        "location_home",
        "goals_away",
        "goals_home",
        "season_away",
        "date_away",
        "location_away",
        "location_away",
        "teamID_x",
        "teamID_y",
    ],
    axis=1,
)


# df_appearances = pd.read_csv("appearances.csv")
# df_players = pd.read_csv("players.csv", encoding="ISO-8859-1")
# df_shots = pd.read_csv("shots.csv")

df_apperance_merged_with_players = pd.merge(
    df_appearances, df_players, on="playerID"
)





In [24]:
df2_appearances = pd.read_csv("players/appearances.csv")
df2_club_games = pd.read_csv("players/club_games.csv")
df2_clubs = pd.read_csv("players/clubs.csv")
df2_competitions = pd.read_csv("players/competitions.csv")
df2_games = pd.read_csv("players/games.csv")
df2_player_val= pd.read_csv("players/player_valuations.csv")
df2_players = pd.read_csv("players/players.csv")
df2_players.info()


""" df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")
df2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")
df2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")
df2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")
df2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")
df2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")
df2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") """


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25619 entries, 0 to 25618
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    25619 non-null  int64  
 1   pretty_name                  25619 non-null  object 
 2   club_id                      25619 non-null  int64  
 3   club_pretty_name             25619 non-null  object 
 4   current_club_id              25619 non-null  int64  
 5   country_of_citizenship       25618 non-null  object 
 6   country_of_birth             23903 non-null  object 
 7   date_of_birth                25593 non-null  object 
 8   position                     25619 non-null  object 
 9   sub_position                 22544 non-null  object 
 10  name                         25619 non-null  object 
 11  foot                         23727 non-null  object 
 12  height_in_cm                 25619 non-null  int64  
 13  market_value_in_

' df2_appearances = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/appearances.csv")\ndf2_club_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/club_games.csv")\ndf2_clubs = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/clubs.csv")\ndf2_competitions = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/competitions.csv")\ndf2_games = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/games.csv")\ndf2_player_val= pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/player_valuations.csv")\ndf2_players = pd.read_csv("https://raw.githubusercontent.com/campeone-pixel/soccer_database/master/players/players.csv") '

In [25]:

top_5_ligas=df2_competitions.loc[(df2_competitions['pretty_name']=='Bundesliga') | (df2_competitions['pretty_name']=='Laliga') | (df2_competitions['pretty_name']=='Premier League') | (df2_competitions['pretty_name']=='Serie A') | (df2_competitions['pretty_name']=='Ligue 1') ]
#!pip install thefuzz
#!pip install rapidfuzz
#!pip install python-Levenshtein
df_f_game[['date','hour']]=df_f_game['date'].str.split(' ',1,expand=True)
pd.options.mode.chained_assignment = None  # default='warn'

from fuzzywuzzy import fuzz, process


from thefuzz import process, fuzz

def match_teams(df_x,df_match,score):

 


  match=process.extractOne(df_x.name, df_m_filter.name,score_cutoff=score,scorer=fuzz.WRatio)
  
  
  if match:
    return match[2]
  else:
    return 2

#---------------------------------------------------------------------------------------------------------
# top_5_ligas["pretty_name"]      df_teams.name
#top_5_ligas['new_id'] = df_teams.loc[top_5_ligas["pretty_name"].apply(lambda x: match(x,df_teams.name,70) ).values,"name"].values
#---------------------------------------------------------------------------------------------------------

seasons=df_f_game["season"].unique()
df2_games= df2_games[df2_games['season'].isin(seasons)]
df2_games_top5= df2_games[df2_games['competition_id'].isin(top_5_ligas['competition_id'])]
df2_clubs_top5=df2_clubs[df2_clubs['domestic_competition_id'].isin(top_5_ligas['competition_id'])]

#---------------------------------------------------------------------------------------------------------
# df2_clubs_top5['pretty_name']   df_teams.name
#df2_clubs_top5['new_id'] = df_teams.loc[df2_clubs_top5.apply(lambda x: match(x,df_teams,80) ).values,"teamID"].values
#---------------------------------------------------------------------------------------------------------


df2_appearances_top5= df2_appearances[df2_appearances['game_id'].isin(df2_games_top5['game_id'])]
df2_players_top5=df2_players[df2_players['current_club_id'].isin(df2_clubs_top5['club_id'])]


#-----------------------------------------------------------------------------------------
# df2_players_top5['pretty_name']      df_players.name
#df2_players_top5['new_id'] = df_players.loc[df2_players_top5.apply(lambda x: match(x,df_players,70) ).values,"name"].values
#----------------------------------------------------------------------------------------------


df2_players_val_top5=df2_player_val[df2_player_val['player_id'].isin(df2_players_top5['player_id'])]






In [31]:
lista_df_teams=df_teams.columns
lista_df2_clubs=df2_clubs.columns
df_f_game_col=df_f_game.columns
df2_games_top5_col=df2_games_top5.columns
print(df2_games_top5_col)
print(df_f_game_col)

Index(['game_id', 'competition_id', 'competition_type', 'season', 'round',
       'date', 'home_club_id', 'away_club_id', 'home_club_goals',
       'away_club_goals', 'aggregate', 'home_club_position',
       'away_club_position', 'club_home_pretty_name', 'club_away_pretty_name',
       'home_club_manager_name', 'away_club_manager_name', 'stadium',
       'attendance', 'referee', 'url'],
      dtype='object')
Index(['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID',
       'homeGoals', 'awayGoals', 'homeGoalsHalfTime', 'awayGoalsHalfTime',
       'xGoals_home', 'shots_home', 'shotsOnTarget_home', 'deep_home',
       'ppda_home', 'fouls_home', 'corners_home', 'yellowCards_home',
       'redCards_home', 'result_home', 'xGoals_away', 'shots_away',
       'shotsOnTarget_away', 'deep_away', 'ppda_away', 'fouls_away',
       'corners_away', 'yellowCards_away', 'redCards_away', 'result_away',
       'liga', 'home_name', 'away_name', 'hour'],
      dtype='object')


In [34]:
df2_games_top5['match_name']= df2_games_top5['club_home_pretty_name'] +" vs " + df2_games_top5['club_away_pretty_name']
df2_games_top5['match_name']

11336                    Fc Malaga vs Fc Sevilla
11337                   Real Madrid vs Fc Getafe
11342                     Sd Eibar vs Fc Cordoba
11344          Tsg 1899 Hoffenheim vs Hertha Bsc
11346    Borussia Monchengladbach vs Fc Augsburg
                          ...                   
48544                Fc Turin vs Cagliari Calcio
48550               Fc Crotone vs Juventus Turin
48559                   Ac Mailand vs Fc Bologna
48613             Benevento Calcio vs Fc Bologna
48619                  Fc Bologna vs Us Sassuolo
Name: match_name, Length: 12681, dtype: object